In [1]:
!pip install torch

In [2]:
!pip install idx2numpy

In [3]:
!pip install sklearn

In [4]:
!pip install pyreadr


In [1]:
# Define random state
random_state = 0

# Import libraries, modules, and functions
import time
import numpy as np
import matplotlib.pyplot as plt

# PyTorch imports
import torch
import torch.nn as nn
import idx2numpy
from sklearn.model_selection import KFold

In [2]:
# Display GPU/CUDA availability
print('\nCUDA availability:  %s\n' % (torch.cuda.is_available()))



CUDA availability:  False



In [3]:
data_path='/Users/ali/Desktop/Jul/apoe2_paper/vertex/NN_sex/' # data path after downloading from mnist website abnd unziping in a folder

In [4]:
import pyreadr
image=pyreadr.read_r(data_path + 'image_filter.rda')
#print(image.keys())
image = image["image_filter"]
#print(image.shape)
sex=pyreadr.read_r(data_path + 'response.rda')
#print(sex.keys())
sex= sex["response"]
print(sex.shape)
print(sex)
sex=sex["Sex"]
print(sex)

(33, 6)
        DWI Genotype              Weight     Sex     Diet  Age_Months
0   57496.0   APOE22                27.5  female  Control       15.57
1   57498.0   APOE22                30.3    male  Control       15.57
2   57500.0   APOE22                  28  female  Control       15.57
3   57502.0   APOE22                29.3    male  Control       15.57
4   57513.0   APOE22  32.299999999999997    male  Control       16.57
5   57552.0   APOE22                  32    male  Control       17.73
6   57692.0   APOE22                  28    male  Control       15.27
7   57694.0   APOE22                22.3  female  Control       15.27
8   57700.0   APOE22                30.3    male  Control       15.27
9   57702.0   APOE22                  24  female  Control       15.27
10  57709.0   APOE22                22.5  female  Control       15.27
11  58344.0   APOE22  33.200000000000003    male  Control       18.73
12  58346.0   APOE22  34.799999999999997    male  Control       18.73
13  58350.0 

In [5]:
sex[sex=="male"]=0;
sex[sex=="female"]=1;
type(sex)
sex

0     1
1     0
2     1
3     0
4     0
5     0
6     0
7     1
8     0
9     1
10    1
11    0
12    0
13    0
14    0
15    0
16    0
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    0
30    0
31    0
32    0
Name: Sex, dtype: object

In [6]:
X_raw_train_npy = image #
y_train_npy = sex

In [7]:
#flatten images
X_raw_train_npy =  np.reshape(X_raw_train_npy, newshape =(X_raw_train_npy.shape[0],-1))

In [8]:
print(X_raw_train_npy.shape)
print(X_raw_train_npy.shape[0])
print(y_train_npy.shape)


(33, 487)
33
(33,)


### store original data

In [9]:
X_raw_train_npy=X_raw_train_npy.to_numpy()
y_train_npy=y_train_npy.to_numpy()

In [10]:
X_raw_train_npy_orig =  X_raw_train_npy
y_train_npy_orig = y_train_npy

In [11]:
y_train_npy = np.array(y_train_npy, dtype=int)
y_train_npy

array([1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [12]:
y_train_npy_orig.shape 

type(y_train_npy_orig)

numpy.ndarray

In [13]:
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
    print( type(X_raw_train_npy_orig[test].shape))
    

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


/Users/ali/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [14]:
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
	print('train: %s, test: %s' % (y_train_npy_orig[train].shape, y_train_npy_orig[test].shape))

train: (26, 487), test: (7, 487)
train: (26,), test: (7,)
train: (26, 487), test: (7, 487)
train: (26,), test: (7,)
train: (26, 487), test: (7, 487)
train: (26,), test: (7,)
train: (27, 487), test: (6, 487)
train: (27,), test: (6,)
train: (27, 487), test: (6, 487)
train: (27,), test: (6,)


In [15]:
y_train_npy_orig[train]

array([1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0], dtype=object)

In [16]:
# MNIST PyTorch dataset class definition
class MNISTDataset(torch.utils.data.Dataset):
    """
    PyTorch dataset object for the MNIST dataset.
    
    Parameters
    ----------
    X : torch.Tensor
        Flattened image data of size (n_samples x 784).
    y : torch.Tensor
        Image labels, integer values 0, 1, 2, ..., 9.
    
    Attributes
    ----------
    X : torch.Tensor
        Flattened image data of size (n_samples x 784).
    y : torch.Tensor
        Image labels, integer values 0, 1, 2, ..., 9.
    len : int
        Length of dataset/number of observations in dataset.
    """
    
    # MNIST PyTorch dataset instantiation method
    def __init__(self, X, y):
        """
        MNIST dataset instantiation method.
        
        Parameters
        ----------
        X : torch.Tensor
            Flattened image data of size (n_samples x 784).
        y : torch.Tensor
            Image labels, integer values 0, 1, 2, ..., 9.
        
        Attributes
        ----------
        X : torch.Tensor
            Flattened image data of size (n_samples x 784).
        y : torch.Tensor
            Image labels, integer values 0, 1, 2, ..., 9.
        len : int
            Length of dataset/number of observations in dataset.
        """
        
        # Assign attributes
        self.X = X         # flattened image data
        self.y = y         # image labels
        self.len = len(X)  # length of dataset

    # Dataset length method
    def __len__(self):
        """
        Method that returns length of dataset/number of observations in dataset.
        
        Returns
        -------
        self.len : int
            Length of dataset/number of observations in dataset.
        """
        
        return self.len

    # Dataset indexing method
    def __getitem__(self, idx):
        """
        Method that retrieves samples and corresponding labels from the dataset.
        
        Parameters
        ----------
        idx : int
            Integer index of dataset sample to be retrieved.
        
        Returns
        -------
        X_samp : torch.Tensor
            Image sample.
        y_samp : torch.Tensor
            Label corresponding to image sample.
        """
        
        # Retrieve sample of data indexed by idx
        X_samp = self.X[idx, :]
        y_samp = self.y[idx]

        # Return data sample
        return X_samp, y_samp


In [17]:
# Logistic regression model class definition
class LogisticRegression(nn.Module):
    """
    PyTorch implementation of a logistic regression model.
    
    Parameters
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.

    Attributes
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    linear : torch.nn.Linear
        Fully-connected/linear layer.
    """
    
    # Logistic regression instantiation method
    def __init__(self, input_dim, output_dim):
        """
        Logistic regression model instantiation method.
        
        Parameters
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.

        Attributes
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        linear : torch.nn.Linear
            Fully-connected/linear layer.
        """
        
        # Inherit from torch.nn.Module
        super(LogisticRegression, self).__init__()
        
        # Assign logistic regression parameters to model attributes
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.linear = nn.Linear(
            in_features=input_dim,
            out_features=output_dim,
            bias=True)
    
    # Logistic regression forward pass method
    def forward(self, x):
        """
        Logistic regression forward pass method.
        
        Parameters
        ----------
        x : torch.Tensor()
            Tensor of input data.
        
        Returns
        -------
        logits : torch.Tensor()
             Raw model predictions (not passed through sigmoid or softmax function).
        """
        
        # Calculate model predictions/logits
        logits = self.linear(x)
        
        # Return model output
        return logits


In [18]:
# Instantiate cross-entropy loss function
loss_func = nn.CrossEntropyLoss()


In [19]:
# Number of training epochs (epochs = # of passes through data)
n_epochs = 500

In [20]:
import random
torch.manual_seed(0)
np.random.seed(0)

In [21]:

testkfold=[]
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
#	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
    X_raw_train=X_raw_train_npy_orig[train]
    X_raw_test=X_raw_train_npy_orig[test]
    y_train=np.array(y_train_npy_orig[train] , dtype=float)
    y_test=np.array(y_train_npy_orig[test], dtype=float)
    #y_train = torch.tensor(y_train, dtype=torch.long)
    #y_test = torch.tensor(y_test, dtype=torch.long)


    
    
    X_raw_train = torch.from_numpy(X_raw_train) #format torch data
    y_train = torch.from_numpy(y_train)
    X_raw_test = torch.from_numpy(X_raw_test)
    y_test = torch.from_numpy(y_test)

    X_raw_train = X_raw_train.float() #float
    X_raw_test = X_raw_test.float()
    
    #X_train = X_raw_train / 255.0 #scale
    #X_test = X_raw_test / 255.0
    X_train=X_raw_train
    X_test=X_raw_test
    
    
    train_dataset = MNISTDataset(X_train, y_train) #pytorch data
    test_dataset = MNISTDataset(X_test, y_test)
    
    batch_size = 10
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    input_dim = X_train.shape[1]             #Define model input and output dimensions from data
    output_dim = len(np.unique(y_train.numpy()))  

    
    lr_model = LogisticRegression(input_dim=input_dim, output_dim=output_dim)# Instantiate logistic regression model
    
    lr = 0.1# Instantiate stochastic gradient descent (SGD) optimizer
    
    optimizer = torch.optim.SGD(lr_model.parameters(), lr=lr)
    #optimizer = torch.optim.Adam(lr_model.parameters())
    
    print('\nTraining logistic regression model...\n')# Time model training
    time_start = time.time()
    
    lr_model.train()# train() method affects operations such as dropout and batch normalization
        # Train model/iterate over epochs
    for epoch in range(n_epochs):
        # Initialize epoch metrics variables
        n_obs = 0
        loss_sum = 0
        n_correct = 0

        # Iterate through training data mini-batchesbatches
        for images_batch, labels_batch in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = lr_model(images_batch)         # model predictions
            loss = loss_func(y_pred, labels_batch.long())  # loss function evaluation

            # Backward pass
            loss.backward()    # backpropagation
            optimizer.step()   # update parameters according to learning rate, gradients

            # Update epoch metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            loss_sum += n_batch * loss.item()
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

        # Calculate epoch training loss and training accuracy
        loss_train = loss_sum / n_obs
        acc_train = n_correct / n_obs

        # Display training progress
        prog_disp_freq = 1   # frequency of training progress display
        if (epoch + 1) % prog_disp_freq == 0 or epoch == 0 or (epoch + 1) == n_epochs:
            print('  E%02d | train loss: %s | train acc.: %s' % 
                  (epoch + 1, '{:.4f}'.format(loss_train), '{:.4f}'.format(acc_train)))

    # Time model training
    time_end = time.time()
    print('\nLogistic regression model training complete.\n')
    print('Time to train logistic regression model:  %.1f s\n' % (time_end - time_start))
        # Place model in evaluation mode
    # .eval() method affects operations such as dropout and batch normalization
    lr_model.eval()

    # Initialize test set metrics variables
    n_obs = 0
    n_correct = 0

    # Disable gradient calculation
    with torch.no_grad():
        # Iterate through test data mini-batches
        for images_batch, labels_batch in test_loader:
            # Forward pass
            y_pred = lr_model(images_batch)  # model predictions

            # Update test set metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

    # Calculate test accuracy
    acc_test = n_correct / n_obs

    # Display test accuracy
    print('\nLogistic regression model MNIST test acc.:  %.4f\n' % (acc_test))
    testkfold.append(acc_test)



Training logistic regression model...

  E01 | train loss: 0.4467 | train acc.: 0.7692
  E02 | train loss: 0.0242 | train acc.: 1.0000
  E03 | train loss: 0.0098 | train acc.: 1.0000
  E04 | train loss: 0.0082 | train acc.: 1.0000
  E05 | train loss: 0.0071 | train acc.: 1.0000
  E06 | train loss: 0.0062 | train acc.: 1.0000
  E07 | train loss: 0.0056 | train acc.: 1.0000
  E08 | train loss: 0.0051 | train acc.: 1.0000
  E09 | train loss: 0.0047 | train acc.: 1.0000
  E10 | train loss: 0.0043 | train acc.: 1.0000
  E11 | train loss: 0.0040 | train acc.: 1.0000
  E12 | train loss: 0.0038 | train acc.: 1.0000
  E13 | train loss: 0.0035 | train acc.: 1.0000
  E14 | train loss: 0.0034 | train acc.: 1.0000
  E15 | train loss: 0.0032 | train acc.: 1.0000
  E16 | train loss: 0.0030 | train acc.: 1.0000
  E17 | train loss: 0.0029 | train acc.: 1.0000
  E18 | train loss: 0.0027 | train acc.: 1.0000
  E19 | train loss: 0.0026 | train acc.: 1.0000
  E20 | train loss: 0.0025 | train acc.: 1.0000


  E314 | train loss: 0.0002 | train acc.: 1.0000
  E315 | train loss: 0.0002 | train acc.: 1.0000
  E316 | train loss: 0.0002 | train acc.: 1.0000
  E317 | train loss: 0.0002 | train acc.: 1.0000
  E318 | train loss: 0.0002 | train acc.: 1.0000
  E319 | train loss: 0.0002 | train acc.: 1.0000
  E320 | train loss: 0.0002 | train acc.: 1.0000
  E321 | train loss: 0.0002 | train acc.: 1.0000
  E322 | train loss: 0.0002 | train acc.: 1.0000
  E323 | train loss: 0.0002 | train acc.: 1.0000
  E324 | train loss: 0.0002 | train acc.: 1.0000
  E325 | train loss: 0.0002 | train acc.: 1.0000
  E326 | train loss: 0.0002 | train acc.: 1.0000
  E327 | train loss: 0.0002 | train acc.: 1.0000
  E328 | train loss: 0.0002 | train acc.: 1.0000
  E329 | train loss: 0.0002 | train acc.: 1.0000
  E330 | train loss: 0.0002 | train acc.: 1.0000
  E331 | train loss: 0.0002 | train acc.: 1.0000
  E332 | train loss: 0.0002 | train acc.: 1.0000
  E333 | train loss: 0.0002 | train acc.: 1.0000
  E334 | train loss:

  E492 | train loss: 0.0001 | train acc.: 1.0000
  E493 | train loss: 0.0001 | train acc.: 1.0000
  E494 | train loss: 0.0001 | train acc.: 1.0000
  E495 | train loss: 0.0001 | train acc.: 1.0000
  E496 | train loss: 0.0001 | train acc.: 1.0000
  E497 | train loss: 0.0001 | train acc.: 1.0000
  E498 | train loss: 0.0001 | train acc.: 1.0000
  E499 | train loss: 0.0001 | train acc.: 1.0000
  E500 | train loss: 0.0001 | train acc.: 1.0000

Logistic regression model training complete.

Time to train logistic regression model:  0.6 s


Logistic regression model MNIST test acc.:  1.0000


Training logistic regression model...

  E01 | train loss: 0.4980 | train acc.: 0.7308
  E02 | train loss: 0.0311 | train acc.: 1.0000
  E03 | train loss: 0.0137 | train acc.: 1.0000
  E04 | train loss: 0.0099 | train acc.: 1.0000
  E05 | train loss: 0.0080 | train acc.: 1.0000
  E06 | train loss: 0.0067 | train acc.: 1.0000
  E07 | train loss: 0.0058 | train acc.: 1.0000
  E08 | train loss: 0.0052 | train

  E165 | train loss: 0.0003 | train acc.: 1.0000
  E166 | train loss: 0.0003 | train acc.: 1.0000
  E167 | train loss: 0.0003 | train acc.: 1.0000
  E168 | train loss: 0.0003 | train acc.: 1.0000
  E169 | train loss: 0.0003 | train acc.: 1.0000
  E170 | train loss: 0.0003 | train acc.: 1.0000
  E171 | train loss: 0.0003 | train acc.: 1.0000
  E172 | train loss: 0.0003 | train acc.: 1.0000
  E173 | train loss: 0.0003 | train acc.: 1.0000
  E174 | train loss: 0.0003 | train acc.: 1.0000
  E175 | train loss: 0.0003 | train acc.: 1.0000
  E176 | train loss: 0.0003 | train acc.: 1.0000
  E177 | train loss: 0.0003 | train acc.: 1.0000
  E178 | train loss: 0.0003 | train acc.: 1.0000
  E179 | train loss: 0.0003 | train acc.: 1.0000
  E180 | train loss: 0.0003 | train acc.: 1.0000
  E181 | train loss: 0.0003 | train acc.: 1.0000
  E182 | train loss: 0.0003 | train acc.: 1.0000
  E183 | train loss: 0.0003 | train acc.: 1.0000
  E184 | train loss: 0.0003 | train acc.: 1.0000
  E185 | train loss:

  E341 | train loss: 0.0002 | train acc.: 1.0000
  E342 | train loss: 0.0002 | train acc.: 1.0000
  E343 | train loss: 0.0002 | train acc.: 1.0000
  E344 | train loss: 0.0002 | train acc.: 1.0000
  E345 | train loss: 0.0002 | train acc.: 1.0000
  E346 | train loss: 0.0002 | train acc.: 1.0000
  E347 | train loss: 0.0002 | train acc.: 1.0000
  E348 | train loss: 0.0002 | train acc.: 1.0000
  E349 | train loss: 0.0002 | train acc.: 1.0000
  E350 | train loss: 0.0002 | train acc.: 1.0000
  E351 | train loss: 0.0002 | train acc.: 1.0000
  E352 | train loss: 0.0002 | train acc.: 1.0000
  E353 | train loss: 0.0002 | train acc.: 1.0000
  E354 | train loss: 0.0002 | train acc.: 1.0000
  E355 | train loss: 0.0002 | train acc.: 1.0000
  E356 | train loss: 0.0002 | train acc.: 1.0000
  E357 | train loss: 0.0002 | train acc.: 1.0000
  E358 | train loss: 0.0002 | train acc.: 1.0000
  E359 | train loss: 0.0002 | train acc.: 1.0000
  E360 | train loss: 0.0002 | train acc.: 1.0000
  E361 | train loss:

...

  E203 | train loss: 0.0003 | train acc.: 1.0000
  E204 | train loss: 0.0003 | train acc.: 1.0000
  E205 | train loss: 0.0003 | train acc.: 1.0000
  E206 | train loss: 0.0003 | train acc.: 1.0000
  E207 | train loss: 0.0003 | train acc.: 1.0000
  E208 | train loss: 0.0003 | train acc.: 1.0000
  E209 | train loss: 0.0003 | train acc.: 1.0000
  E210 | train loss: 0.0003 | train acc.: 1.0000
  E211 | train loss: 0.0003 | train acc.: 1.0000
  E212 | train loss: 0.0003 | train acc.: 1.0000
  E213 | train loss: 0.0003 | train acc.: 1.0000
  E214 | train loss: 0.0003 | train acc.: 1.0000
  E215 | train loss: 0.0003 | train acc.: 1.0000
  E216 | train loss: 0.0003 | train acc.: 1.0000
  E217 | train loss: 0.0003 | train acc.: 1.0000
  E218 | train loss: 0.0003 | train acc.: 1.0000
  E219 | train loss: 0.0003 | train acc.: 1.0000
  E220 | train loss: 0.0003 | train acc.: 1.0000
  E221 | train loss: 0.0003 | train acc.: 1.0000
  E222 | train loss: 0.0003 | train acc.: 1.0000
  E223 | train loss:

  E377 | train loss: 0.0002 | train acc.: 1.0000
  E378 | train loss: 0.0002 | train acc.: 1.0000
  E379 | train loss: 0.0002 | train acc.: 1.0000
  E380 | train loss: 0.0002 | train acc.: 1.0000
  E381 | train loss: 0.0002 | train acc.: 1.0000
  E382 | train loss: 0.0002 | train acc.: 1.0000
  E383 | train loss: 0.0002 | train acc.: 1.0000
  E384 | train loss: 0.0002 | train acc.: 1.0000
  E385 | train loss: 0.0002 | train acc.: 1.0000
  E386 | train loss: 0.0002 | train acc.: 1.0000
  E387 | train loss: 0.0002 | train acc.: 1.0000
  E388 | train loss: 0.0002 | train acc.: 1.0000
  E389 | train loss: 0.0002 | train acc.: 1.0000
  E390 | train loss: 0.0002 | train acc.: 1.0000
  E391 | train loss: 0.0002 | train acc.: 1.0000
  E392 | train loss: 0.0002 | train acc.: 1.0000
  E393 | train loss: 0.0002 | train acc.: 1.0000
  E394 | train loss: 0.0002 | train acc.: 1.0000
  E395 | train loss: 0.0002 | train acc.: 1.0000
  E396 | train loss: 0.0002 | train acc.: 1.0000
  E397 | train loss:

  E51 | train loss: 0.0008 | train acc.: 1.0000
  E52 | train loss: 0.0008 | train acc.: 1.0000
  E53 | train loss: 0.0008 | train acc.: 1.0000
  E54 | train loss: 0.0008 | train acc.: 1.0000
  E55 | train loss: 0.0008 | train acc.: 1.0000
  E56 | train loss: 0.0008 | train acc.: 1.0000
  E57 | train loss: 0.0008 | train acc.: 1.0000
  E58 | train loss: 0.0008 | train acc.: 1.0000
  E59 | train loss: 0.0007 | train acc.: 1.0000
  E60 | train loss: 0.0007 | train acc.: 1.0000
  E61 | train loss: 0.0007 | train acc.: 1.0000
  E62 | train loss: 0.0007 | train acc.: 1.0000
  E63 | train loss: 0.0007 | train acc.: 1.0000
  E64 | train loss: 0.0007 | train acc.: 1.0000
  E65 | train loss: 0.0007 | train acc.: 1.0000
  E66 | train loss: 0.0007 | train acc.: 1.0000
  E67 | train loss: 0.0007 | train acc.: 1.0000
  E68 | train loss: 0.0007 | train acc.: 1.0000
  E69 | train loss: 0.0007 | train acc.: 1.0000
  E70 | train loss: 0.0006 | train acc.: 1.0000
  E71 | train loss: 0.0006 | train acc.:

  E225 | train loss: 0.0002 | train acc.: 1.0000
  E226 | train loss: 0.0002 | train acc.: 1.0000
  E227 | train loss: 0.0002 | train acc.: 1.0000
  E228 | train loss: 0.0002 | train acc.: 1.0000
  E229 | train loss: 0.0002 | train acc.: 1.0000
  E230 | train loss: 0.0002 | train acc.: 1.0000
  E231 | train loss: 0.0002 | train acc.: 1.0000
  E232 | train loss: 0.0002 | train acc.: 1.0000
  E233 | train loss: 0.0002 | train acc.: 1.0000
  E234 | train loss: 0.0002 | train acc.: 1.0000
  E235 | train loss: 0.0002 | train acc.: 1.0000
  E236 | train loss: 0.0002 | train acc.: 1.0000
  E237 | train loss: 0.0002 | train acc.: 1.0000
  E238 | train loss: 0.0002 | train acc.: 1.0000
  E239 | train loss: 0.0002 | train acc.: 1.0000
  E240 | train loss: 0.0002 | train acc.: 1.0000
  E241 | train loss: 0.0002 | train acc.: 1.0000
  E242 | train loss: 0.0002 | train acc.: 1.0000
  E243 | train loss: 0.0002 | train acc.: 1.0000
  E244 | train loss: 0.0002 | train acc.: 1.0000
  E245 | train loss:

  E400 | train loss: 0.0001 | train acc.: 1.0000
  E401 | train loss: 0.0001 | train acc.: 1.0000
  E402 | train loss: 0.0001 | train acc.: 1.0000
  E403 | train loss: 0.0001 | train acc.: 1.0000
  E404 | train loss: 0.0001 | train acc.: 1.0000
  E405 | train loss: 0.0001 | train acc.: 1.0000
  E406 | train loss: 0.0001 | train acc.: 1.0000
  E407 | train loss: 0.0001 | train acc.: 1.0000
  E408 | train loss: 0.0001 | train acc.: 1.0000
  E409 | train loss: 0.0001 | train acc.: 1.0000
  E410 | train loss: 0.0001 | train acc.: 1.0000
  E411 | train loss: 0.0001 | train acc.: 1.0000
  E412 | train loss: 0.0001 | train acc.: 1.0000
  E413 | train loss: 0.0001 | train acc.: 1.0000
  E414 | train loss: 0.0001 | train acc.: 1.0000
  E415 | train loss: 0.0001 | train acc.: 1.0000
  E416 | train loss: 0.0001 | train acc.: 1.0000
  E417 | train loss: 0.0001 | train acc.: 1.0000
  E418 | train loss: 0.0001 | train acc.: 1.0000
  E419 | train loss: 0.0001 | train acc.: 1.0000
  E420 | train loss:

In [22]:
np.mean(testkfold)

0.9714285714285715

In [23]:
testkfold

[1.0, 0.8571428571428571, 1.0, 1.0, 1.0]

In [45]:
# Multi-layer perceptron model class definition
class MLP(nn.Module):
    """
    PyTorch implementation of a multi-layer perceptron model.
    
    Parameters
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    hidden_dim : int
        Size/dimensionality of the hidden layer.
    
    Attributes
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    hidden_dim : int
        Size/dimensionality of the hidden layer.
    linear_1 : torch.nn.Linear
        Fully-connected/linear neural network layer.
    linear_2 : torch.nn.Linear
        Fully-connected/linear neural network layer.
    relu : torch.nn.ReLU
        Rectified linear unit activation function.
    """
    
    # MLP instantiation method
    def __init__(self, input_dim, output_dim, hidden_dim=15):
        """
        Model instantiation method.
        
        Parameters
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        hidden_dim : int
            Size/dimensionality of the hidden layer.

        Attributes
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        hidden_dim : int
            Size/dimensionality of the hidden layer.
        linear_1 : torch.nn.Linear
            Fully-connected/linear neural network layer.
        linear_2 : torch.nn.Linear
            Fully-connected/linear neural network layer.
        relu : torch.nn.ReLU
            Rectified linear unit activation function.
        """
        
        # Inherit from torch.nn.Module
        super(MLP, self).__init__()
        
        # Assign MLP parameters to model attributes
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        # Define MLP layers
        self.linear_1 = nn.Linear(
            in_features=input_dim,
            out_features=hidden_dim,
            bias=True)
        self.linear_2 = nn.Linear(
            in_features=hidden_dim,
            out_features=hidden_dim,
            bias=True)
        self.linear_3 = nn.Linear(
            in_features=hidden_dim,
            out_features=output_dim,
            bias=True)
        
        # Define MLP activation function
        self.relu = nn.LeakyReLU()
    
    # MLP forward pass method
    def forward(self, x):
        """
        MLP forward pass method.
        
        Parameters
        ----------
        x : torch.Tensor()
            Tensor of input data.
        
        Returns
        -------
        logits : torch.Tensor()
             Raw model predictions (not passed through sigmoid or softmax function).
        """
        
        # Forward pass through model
        x = self.linear_1(x)       # first fully-connected layer
        x = self.relu(x)           # non-linear activation transformation
        
        s=self.linear_2(x)
        s = self.relu(s)           # non-linear activation transformation

        mids2=self.linear_2(s)
        mids2 = self.relu(mids2)           # non-linear activation transformation
        
        mids3=self.linear_2(mids2)
        mids3 = self.relu(mids3)           # non-linear activation transformation

        
        mids4=self.linear_2(mids3)
        mids4 = self.relu(mids4)
        
        mids5=self.linear_2(mids4)
        mids5 = self.relu(mids5)
        
        
        logits = self.linear_3(s)  # second fully-connected layer
        
        # Return model output
        return logits


In [46]:
X_raw_train_npy_orig

array([[-0.12603027,  0.40153656,  3.069269  , ..., -0.4518967 ,
        -0.38450284,  1.00353083],
       [ 3.39178964,  1.8707377 ,  0.13120539, ..., -0.19478306,
        -0.17649311,  1.48494922],
       [ 1.01480624,  0.06678187,  1.83218958, ..., -0.19478306,
        -0.03781995, -0.13667061],
       ...,
       [-0.614135  , -0.45394765, -0.79660417, ..., -0.70901034,
         0.37819951,  1.13021988],
       [ 1.24874992,  0.08227977, -1.10587402, ..., -0.4518967 ,
        -0.10715653, -0.71944024],
       [ 0.0703669 , -0.5407359 ,  0.44047525, ...,  1.86212607,
         3.35967235,  1.89035418]])

In [47]:
import random
random.seed(1)
torch.manual_seed(0)

In [48]:
testkfold=[]
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
#	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
    X_raw_train=X_raw_train_npy_orig[train]
    X_raw_test=X_raw_train_npy_orig[test]
    y_train=np.array(y_train_npy_orig[train] , dtype=float)
    y_test=np.array(y_train_npy_orig[test], dtype=float)
    
    X_raw_train = torch.from_numpy(X_raw_train) #format torch data
    y_train = torch.from_numpy(y_train)
    X_raw_test = torch.from_numpy(X_raw_test)
    y_test = torch.from_numpy(y_test)

    X_raw_train = X_raw_train.float() #float
    X_raw_test = X_raw_test.float()
    
    #X_train = X_raw_train / 255.0 #scale
    #X_test = X_raw_test / 255.0
    X_train = X_raw_train 
    X_test = X_raw_test
    
    train_dataset = MNISTDataset(X_train, y_train) #pytorch data
    test_dataset = MNISTDataset(X_test, y_test)
    
    batch_size = 16
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
        # Define model input and output dimensions from data
    input_dim = X_train.shape[1]                  # second dimension is data dimensionality
    output_dim = len(np.unique(y_train.numpy()))  # output dim. is equal to number of unique labels

    # Instantiate multi-layer perceptron model
    mlp_model = MLP(input_dim=input_dim, output_dim=output_dim)

    # Instantiate stochastic gradient descent (SGD) optimizer
    lr = 0.05
    optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)
    #optimizer = torch.optim.Adam(mlp_model.parameters())

    
    print('\nTraining logistic regression model...\n')# Time model training
    time_start = time.time()

    # Place model in training mode
    # .train() method affects operations such as dropout and batch normalization
    mlp_model.train()

    # Train model/iterate over epochs
    for epoch in range(n_epochs):
        # Initialize epoch metrics variables
        n_obs = 0
        loss_sum = 0
        n_correct = 0

        # Iterate through training data mini-batches
        for images_batch, labels_batch in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = mlp_model(images_batch)         # model predictions
            loss = loss_func(y_pred, labels_batch.long())  # loss function evaluation

            # Backward pass
            loss.backward()    # backpropagation
            optimizer.step()   # update parameters according to learning rate, gradients

            # Update epoch metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            loss_sum += n_batch * loss.item()
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

        # Calculate epoch training loss and training accuracy
        loss_train = loss_sum / n_obs
        acc_train = n_correct / n_obs

        # Display training progress
        prog_disp_freq = 1   # frequency of training progress display
        if (epoch + 1) % prog_disp_freq == 0 or epoch == 0 or epoch + 1 == n_epochs:
            print('  E%02d | train loss: %s | train acc.: %s' % 
                  (epoch + 1, '{:.4f}'.format(loss_train), '{:.4f}'.format(acc_train)))

    # Time model training
    time_end = time.time()
    print('\nMLP model training complete.\n')
    print('Time to train MLP model:  %.1f s\n' % (time_end - time_start))
    # Place model in evaluation mode
    # .eval() method affects operations such as dropout and batch normalization
    mlp_model.eval()

    # Initialize test set metrics variables
    n_obs = 0
    n_correct = 0

    # Disable gradient calculation
    with torch.no_grad():
        # Iterate through test data mini-batches
        for images_batch, labels_batch in test_loader:
            # Forward pass
            y_pred = mlp_model(images_batch)  # model predictions

            # Update test set metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

    # Calculate test accuracy
    acc_test = n_correct / n_obs

    # Display test accuracy
    #print('\nMLP model MNIST test acc.:  %.4f\n' % (acc_test))
    testkfold.append(acc_test)



Training logistic regression model...

  E01 | train loss: 0.6768 | train acc.: 0.6538
  E02 | train loss: 0.6538 | train acc.: 0.8077
  E03 | train loss: 0.6361 | train acc.: 0.8077
  E04 | train loss: 0.6165 | train acc.: 0.8077
  E05 | train loss: 0.5958 | train acc.: 0.9231
  E06 | train loss: 0.5751 | train acc.: 0.9231
  E07 | train loss: 0.5513 | train acc.: 0.9231
  E08 | train loss: 0.5263 | train acc.: 0.9231
  E09 | train loss: 0.4987 | train acc.: 0.9231
  E10 | train loss: 0.4693 | train acc.: 0.9231
  E11 | train loss: 0.4378 | train acc.: 0.9231
  E12 | train loss: 0.4053 | train acc.: 0.9231
  E13 | train loss: 0.3712 | train acc.: 0.9231
  E14 | train loss: 0.3384 | train acc.: 0.9615
  E15 | train loss: 0.3069 | train acc.: 0.9615
  E16 | train loss: 0.2774 | train acc.: 0.9615
  E17 | train loss: 0.2470 | train acc.: 1.0000
  E18 | train loss: 0.2207 | train acc.: 1.0000
  E19 | train loss: 0.1959 | train acc.: 1.0000
  E20 | train loss: 0.1746 | train acc.: 1.0000


  E313 | train loss: 0.0006 | train acc.: 1.0000
  E314 | train loss: 0.0006 | train acc.: 1.0000
  E315 | train loss: 0.0006 | train acc.: 1.0000
  E316 | train loss: 0.0006 | train acc.: 1.0000
  E317 | train loss: 0.0006 | train acc.: 1.0000
  E318 | train loss: 0.0006 | train acc.: 1.0000
  E319 | train loss: 0.0006 | train acc.: 1.0000
  E320 | train loss: 0.0006 | train acc.: 1.0000
  E321 | train loss: 0.0006 | train acc.: 1.0000
  E322 | train loss: 0.0006 | train acc.: 1.0000
  E323 | train loss: 0.0006 | train acc.: 1.0000
  E324 | train loss: 0.0006 | train acc.: 1.0000
  E325 | train loss: 0.0006 | train acc.: 1.0000
  E326 | train loss: 0.0006 | train acc.: 1.0000
  E327 | train loss: 0.0005 | train acc.: 1.0000
  E328 | train loss: 0.0005 | train acc.: 1.0000
  E329 | train loss: 0.0005 | train acc.: 1.0000
  E330 | train loss: 0.0005 | train acc.: 1.0000
  E331 | train loss: 0.0005 | train acc.: 1.0000
  E332 | train loss: 0.0005 | train acc.: 1.0000
  E333 | train loss:

  E131 | train loss: 0.0027 | train acc.: 1.0000
  E132 | train loss: 0.0026 | train acc.: 1.0000
  E133 | train loss: 0.0026 | train acc.: 1.0000
  E134 | train loss: 0.0026 | train acc.: 1.0000
  E135 | train loss: 0.0025 | train acc.: 1.0000
  E136 | train loss: 0.0025 | train acc.: 1.0000
  E137 | train loss: 0.0025 | train acc.: 1.0000
  E138 | train loss: 0.0024 | train acc.: 1.0000
  E139 | train loss: 0.0024 | train acc.: 1.0000
  E140 | train loss: 0.0024 | train acc.: 1.0000
  E141 | train loss: 0.0023 | train acc.: 1.0000
  E142 | train loss: 0.0023 | train acc.: 1.0000
  E143 | train loss: 0.0023 | train acc.: 1.0000
  E144 | train loss: 0.0022 | train acc.: 1.0000
  E145 | train loss: 0.0022 | train acc.: 1.0000
  E146 | train loss: 0.0022 | train acc.: 1.0000
  E147 | train loss: 0.0021 | train acc.: 1.0000
  E148 | train loss: 0.0021 | train acc.: 1.0000
  E149 | train loss: 0.0021 | train acc.: 1.0000
  E150 | train loss: 0.0021 | train acc.: 1.0000
  E151 | train loss:

  E449 | train loss: 0.0004 | train acc.: 1.0000
  E450 | train loss: 0.0004 | train acc.: 1.0000
  E451 | train loss: 0.0004 | train acc.: 1.0000
  E452 | train loss: 0.0004 | train acc.: 1.0000
  E453 | train loss: 0.0004 | train acc.: 1.0000
  E454 | train loss: 0.0004 | train acc.: 1.0000
  E455 | train loss: 0.0004 | train acc.: 1.0000
  E456 | train loss: 0.0004 | train acc.: 1.0000
  E457 | train loss: 0.0004 | train acc.: 1.0000
  E458 | train loss: 0.0004 | train acc.: 1.0000
  E459 | train loss: 0.0003 | train acc.: 1.0000
  E460 | train loss: 0.0003 | train acc.: 1.0000
  E461 | train loss: 0.0003 | train acc.: 1.0000
  E462 | train loss: 0.0003 | train acc.: 1.0000
  E463 | train loss: 0.0003 | train acc.: 1.0000
  E464 | train loss: 0.0003 | train acc.: 1.0000
  E465 | train loss: 0.0003 | train acc.: 1.0000
  E466 | train loss: 0.0003 | train acc.: 1.0000
  E467 | train loss: 0.0003 | train acc.: 1.0000
  E468 | train loss: 0.0003 | train acc.: 1.0000
  E469 | train loss:

  E266 | train loss: 0.0007 | train acc.: 1.0000
  E267 | train loss: 0.0007 | train acc.: 1.0000
  E268 | train loss: 0.0007 | train acc.: 1.0000
  E269 | train loss: 0.0007 | train acc.: 1.0000
  E270 | train loss: 0.0007 | train acc.: 1.0000
  E271 | train loss: 0.0007 | train acc.: 1.0000
  E272 | train loss: 0.0007 | train acc.: 1.0000
  E273 | train loss: 0.0007 | train acc.: 1.0000
  E274 | train loss: 0.0007 | train acc.: 1.0000
  E275 | train loss: 0.0007 | train acc.: 1.0000
  E276 | train loss: 0.0007 | train acc.: 1.0000
  E277 | train loss: 0.0007 | train acc.: 1.0000
  E278 | train loss: 0.0007 | train acc.: 1.0000
  E279 | train loss: 0.0007 | train acc.: 1.0000
  E280 | train loss: 0.0007 | train acc.: 1.0000
  E281 | train loss: 0.0007 | train acc.: 1.0000
  E282 | train loss: 0.0007 | train acc.: 1.0000
  E283 | train loss: 0.0007 | train acc.: 1.0000
  E284 | train loss: 0.0007 | train acc.: 1.0000
  E285 | train loss: 0.0007 | train acc.: 1.0000
  E286 | train loss:

  E81 | train loss: 0.0246 | train acc.: 1.0000
  E82 | train loss: 0.0244 | train acc.: 1.0000
  E83 | train loss: 0.0242 | train acc.: 1.0000
  E84 | train loss: 0.0240 | train acc.: 1.0000
  E85 | train loss: 0.0239 | train acc.: 1.0000
  E86 | train loss: 0.0237 | train acc.: 1.0000
  E87 | train loss: 0.0235 | train acc.: 1.0000
  E88 | train loss: 0.0233 | train acc.: 1.0000
  E89 | train loss: 0.0232 | train acc.: 1.0000
  E90 | train loss: 0.0231 | train acc.: 1.0000
  E91 | train loss: 0.0229 | train acc.: 1.0000
  E92 | train loss: 0.0228 | train acc.: 1.0000
  E93 | train loss: 0.0226 | train acc.: 1.0000
  E94 | train loss: 0.0225 | train acc.: 1.0000
  E95 | train loss: 0.0223 | train acc.: 1.0000
  E96 | train loss: 0.0222 | train acc.: 1.0000
  E97 | train loss: 0.0220 | train acc.: 1.0000
  E98 | train loss: 0.0219 | train acc.: 1.0000
  E99 | train loss: 0.0218 | train acc.: 1.0000
  E100 | train loss: 0.0216 | train acc.: 1.0000
  E101 | train loss: 0.0215 | train acc

  E403 | train loss: 0.0007 | train acc.: 1.0000
  E404 | train loss: 0.0007 | train acc.: 1.0000
  E405 | train loss: 0.0007 | train acc.: 1.0000
  E406 | train loss: 0.0007 | train acc.: 1.0000
  E407 | train loss: 0.0007 | train acc.: 1.0000
  E408 | train loss: 0.0007 | train acc.: 1.0000
  E409 | train loss: 0.0007 | train acc.: 1.0000
  E410 | train loss: 0.0007 | train acc.: 1.0000
  E411 | train loss: 0.0007 | train acc.: 1.0000
  E412 | train loss: 0.0007 | train acc.: 1.0000
  E413 | train loss: 0.0007 | train acc.: 1.0000
  E414 | train loss: 0.0007 | train acc.: 1.0000
  E415 | train loss: 0.0007 | train acc.: 1.0000
  E416 | train loss: 0.0007 | train acc.: 1.0000
  E417 | train loss: 0.0007 | train acc.: 1.0000
  E418 | train loss: 0.0007 | train acc.: 1.0000
  E419 | train loss: 0.0007 | train acc.: 1.0000
  E420 | train loss: 0.0007 | train acc.: 1.0000
  E421 | train loss: 0.0007 | train acc.: 1.0000
  E422 | train loss: 0.0007 | train acc.: 1.0000
  E423 | train loss:

  E222 | train loss: 0.0011 | train acc.: 1.0000
  E223 | train loss: 0.0011 | train acc.: 1.0000
  E224 | train loss: 0.0011 | train acc.: 1.0000
  E225 | train loss: 0.0010 | train acc.: 1.0000
  E226 | train loss: 0.0010 | train acc.: 1.0000
  E227 | train loss: 0.0010 | train acc.: 1.0000
  E228 | train loss: 0.0010 | train acc.: 1.0000
  E229 | train loss: 0.0010 | train acc.: 1.0000
  E230 | train loss: 0.0010 | train acc.: 1.0000
  E231 | train loss: 0.0010 | train acc.: 1.0000
  E232 | train loss: 0.0010 | train acc.: 1.0000
  E233 | train loss: 0.0010 | train acc.: 1.0000
  E234 | train loss: 0.0010 | train acc.: 1.0000
  E235 | train loss: 0.0010 | train acc.: 1.0000
  E236 | train loss: 0.0010 | train acc.: 1.0000
  E237 | train loss: 0.0010 | train acc.: 1.0000
  E238 | train loss: 0.0010 | train acc.: 1.0000
  E239 | train loss: 0.0010 | train acc.: 1.0000
  E240 | train loss: 0.0010 | train acc.: 1.0000
  E241 | train loss: 0.0009 | train acc.: 1.0000
  E242 | train loss:

In [49]:
np.mean(testkfold)

0.8142857142857143

In [50]:
testkfold

[1.0, 0.5714285714285714, 1.0, 0.8333333333333334, 0.6666666666666666]

In [30]:
#X_raw_test